# Хакатон: Извлечение именованных сущностей (NER)
Задача: построить модель для извлечения сущностей из поисковых запросов.

-Сущности: TYPE, BRAND, VOLUME, PERCENT

-Формат: BIO-разметка

-Метрика: macro-averaged F1

-Ограничение: время ответа ≤ 1 секунды




---



# 1: Настройка и Подготовка Данных

## 1.1: Импорт библиотек

In [1]:
import pandas as pd
import ast
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from tabulate import tabulate

In [2]:
# === УСТАНОВКА ДОПОЛНИТЕЛЬНЫХ БИБЛИОТЕК ===
!pip install evaluate seqeval
import evaluate

## 1.2: Загрузка и первичный анализ данных

In [6]:
data_frame = pd.read_csv("train.csv", sep=";")
data_frame_test = pd.read_csv("submission.csv", sep=";")

data_frame.head()


,sample,annotation
0,aa,"[(0, 2, 'O')]"
1,aala,"[(0, 4, 'O')]"
2,aarcca,"[(0, 6, 'O')]"
3,abon,"[(0, 4, 'O')]"
4,abso,"[(0, 4, 'B-BRAND')]"


## 1.3: BIO-разметка и словари меток

In [7]:
labels = [
    "O", "B-BRAND", "I-BRAND", "B-TYPE", "I-TYPE",
    "B-PERCENT", "I-PERCENT", "B-VOLUME", "I-VOLUME"
]

label_to_id = {label: idx for idx, label in enumerate(labels)}
id_to_label = {idx: label for idx, label in enumerate(labels)}

label_to_id


{'O': 0,
 'B-BRAND': 1,
 'I-BRAND': 2,
 'B-TYPE': 3,
 'I-TYPE': 4,
 'B-PERCENT': 5,
 'I-PERCENT': 6,
 'B-VOLUME': 7,
 'I-VOLUME': 8}

## 1.4: Обработка текста и спанов

In [8]:
def process_text_with_spans(text_content, span_data):
    """Формируем BIO-теги для токенов на основе спанов."""
    character_labels = ["O"] * len(text_content)

    for span_start, span_end, span_tag in span_data:
        span_end = min(span_end, len(text_content))
        for pos in range(span_start, span_end):
            character_labels[pos] = "O" if span_tag == "0" else span_tag

    text_tokens = text_content.split()
    token_tags, current_offset = [], 0

    for token in text_tokens:
        token_length = len(token)
        token_char_labels = character_labels[current_offset:current_offset + token_length]

        current_tag = next((c for c in token_char_labels if c != "O"), "O")
        if current_tag == "0":
            current_tag = "O"

        token_tags.append(current_tag)
        current_offset += token_length + 1  # учитываем пробел

    return {
        "tokens": text_tokens,
        "labels": [label_to_id[tag] for tag in token_tags]
    }




---



# 2: Подготовка датасета и выбор модели

## 2.1: Формирование датасетов

In [9]:
# Тренировочный датасет
training_data = []
for _, row in data_frame.iterrows():
    text_value = row[0]
    span_values = ast.literal_eval(row[1])
    training_data.append(process_text_with_spans(text_value, span_values))

training_dataset = Dataset.from_list(training_data)

# Тестовый датасет
test_data = []
for _, row in data_frame_test.iterrows():
    text_value = row[0]
    span_values = ast.literal_eval(row[1])
    test_data.append(process_text_with_spans(text_value, span_values))

test_dataset = Dataset.from_list(test_data)

# Проверка структуры
print(training_dataset[0])
print(test_dataset[0])


/tmp/ipython-input-3662207086.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text_value = row[0]
/tmp/ipython-input-3662207086.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  span_values = ast.literal_eval(row[1])
/tmp/ipython-input-3662207086.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text_value = row[0]
/tmp/ipython-input-3662207086.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecate

{'tokens': ['aa'], 'labels': [0]}
{'tokens': ['форма', 'для', 'выпечки'], 'labels': [3, 4, 4]}


## 2.2: Конфигурация модели и токенизация

In [10]:
model_identifier = "distilbert-base-multilingual-cased"
text_tokenizer = AutoTokenizer.from_pretrained(model_identifier)

def tokenize_with_alignment(data_examples):
    """Токенизация с выравниванием BIO-меток."""
    tokenized_output = text_tokenizer(
        data_examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=32
    )

    aligned_labels = []
    for idx, label_seq in enumerate(data_examples["labels"]):
        word_indices = tokenized_output.word_ids(batch_index=idx)
        prev_word_idx, label_sequence = None, []

        for word_idx in word_indices:
            if word_idx is None:
                label_sequence.append(-100)
            elif word_idx != prev_word_idx:
                label_sequence.append(label_seq[word_idx])
            else:
                label_sequence.append(label_seq[word_idx])
            prev_word_idx = word_idx

        aligned_labels.append(label_sequence)

    tokenized_output["labels"] = aligned_labels
    return tokenized_output

# Токенизация
tokenized_training_set = training_dataset.map(tokenize_with_alignment, batched=True)
tokenized_test_set = test_dataset.map(tokenize_with_alignment, batched=True)

print(tokenized_training_set[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/27251 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'tokens': ['aa'], 'labels': [-100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 28335, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## 2.3: Разделение на train/validation (90%/10%)

In [11]:
split_dataset = training_dataset.train_test_split(test_size=0.1, seed=42)

tokenized_dataset_split = DatasetDict({
    "train": split_dataset["train"].map(tokenize_with_alignment, batched=True),
    "validation": split_dataset["test"].map(tokenize_with_alignment, batched=True)
})

tokenized_dataset_split

Map:   0%|          | 0/24525 [00:00<?, ? examples/s]

Map:   0%|          | 0/2726 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 24525
    })
    validation: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2726
    })
})



---



# 3: Обучение Модели

## 3.1: Настройка колектора данных и инициализация модели

In [12]:
# Колектор
data_collector = DataCollatorForTokenClassification(text_tokenizer)

In [14]:
# Инициализация модели

from transformers import AutoModelForTokenClassification

labels_count = len(labels)
ner_model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-multilingual-cased",
    num_labels=labels_count,
    id2label=id_to_label,
    label2id=label_to_id
)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3.2: Метрики оценки

In [15]:
evaluation_metric = evaluate.load("seqeval")

def calculate_metrics(prediction_output):
    preds, lbls = prediction_output
    preds = preds.argmax(axis=-1)

    true_lbls = [
        [id_to_label[lbl] for lbl in label_seq if lbl != -100]
        for label_seq in lbls
    ]
    true_preds = [
        [id_to_label[p] for (p, l) in zip(pred_seq, label_seq) if l != -100]
        for pred_seq, label_seq in zip(preds, lbls)
    ]

    metric_results = evaluation_metric.compute(predictions=true_preds, references=true_lbls)
    return {
        "precision": metric_results["overall_precision"],
        "recall": metric_results["overall_recall"],
        "f1": metric_results["overall_f1"],
        "accuracy": metric_results["overall_accuracy"]
    }


## 3.3: Обучение модели

In [17]:
training_parameters = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_trainer = Trainer(
    model=ner_model,
    args=training_parameters,
    train_dataset=tokenized_dataset_split["train"],
    eval_dataset=tokenized_dataset_split["validation"],
    tokenizer=text_tokenizer,
    data_collator=data_collector,
    compute_metrics=calculate_metrics
)



/tmp/ipython-input-2005146601.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  model_trainer = Trainer(


In [18]:
# Запуск обучения
model_trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mldrivetizhin (mldrivetizhin-not-company) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.235600,0.218511,0.938523,0.958725,0.948516,0.935706
2,0.123800,0.235577,0.944724,0.960815,0.952702,0.939888
3,0.094000,0.244740,0.953728,0.969175,0.961389,0.947351
4,0.034300,0.284727,0.955844,0.968130,0.961948,0.948499
5,0.035300,0.330550,0.955790,0.966876,0.961301,0.947351


TrainOutput(global_step=7665, training_loss=0.13598239325419487, metrics={'train_runtime': 773.933, 'train_samples_per_second': 158.444, 'train_steps_per_second': 9.904, 'total_flos': 1001460261816000.0, 'train_loss': 0.13598239325419487, 'epoch': 5.0})

## 3.4: Cохранение модели и создание NER-пайплайна

In [19]:
# === СОХРАНЕНИЕ ОБУЧЕННОЙ МОДЕЛИ ===

model_trainer.save_model("./ner-distilbert")
text_tokenizer.save_pretrained("./ner-distilbert")

('./ner-distilbert/tokenizer_config.json',
 './ner-distilbert/special_tokens_map.json',
 './ner-distilbert/vocab.txt',
 './ner-distilbert/added_tokens.json',
 './ner-distilbert/tokenizer.json')

In [20]:
# === СОЗДАНИЕ NER-ПАЙПЛАЙНА ===
ner_processor = pipeline(
    "token-classification",
    model="./ner-distilbert",
    tokenizer="./ner-distilbert",
    aggregation_strategy="simple"
)

Device set to use cuda:0




---



# 4: Оценка и тестирование

## 4.1: Оценка качества модели

In [21]:
# === ОЦЕНКА КАЧЕСТВА МОДЕЛИ ===

validation_metrics = model_trainer.evaluate()
print(validation_metrics)

{'eval_loss': 0.2847272455692291, 'eval_precision': 0.9558444238109977, 'eval_recall': 0.9681295715778474, 'eval_f1': 0.9619477755282146, 'eval_accuracy': 0.9484992619320978, 'eval_runtime': 2.9538, 'eval_samples_per_second': 922.874, 'eval_steps_per_second': 57.891, 'epoch': 5.0}


In [22]:
# === ТЕСТИРОВАНИЕ НА ТЕСТОВЫХ ДАННЫХ ===

test_metrics = model_trainer.evaluate(tokenized_test_set)
test_metrics

/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.3507627248764038,
 'eval_precision': 0.824207815474191,
 'eval_recall': 0.8914320314865274,
 'eval_f1': 0.8565028943770544,
 'eval_accuracy': 0.8053117162307417,
 'eval_runtime': 5.8377,
 'eval_samples_per_second': 856.502,
 'eval_steps_per_second': 53.617,
 'epoch': 5.0}

## 4.2: Оценка качества в другом формате

In [23]:
def compute_detailed_metrics_table(model_instance, dataset_instance):
    # Получение предсказаний модели
    predictions, labels, _ = model_trainer.predict(dataset_instance)
    predictions = np.argmax(predictions, axis=-1)

    # Обработка предсказаний и истинных меток
    processed_true_labels = []
    processed_predictions = []

    for pred_seq, label_seq in zip(predictions, labels):
        pred_tags = []
        true_tags = []
        for pred_id, true_id in zip(pred_seq, label_seq):
            if true_id == -100:
                continue
            true_tags.append(id_to_label[true_id])
            pred_tags.append(id_to_label[pred_id])
        processed_true_labels.append(true_tags)
        processed_predictions.append(pred_tags)

    # Упрощение меток (удаление B-/I- префиксов)
    flat_true = []
    flat_pred = []
    for true_seq, pred_seq in zip(processed_true_labels, processed_predictions):
        for true_tag, pred_tag in zip(true_seq, pred_seq):
            flat_true.append(true_tag.split("-")[-1] if true_tag != "O" else "O")
            flat_pred.append(pred_tag.split("-")[-1] if pred_tag != "O" else "O")

    # Вычисление метрик по классам
    precision_scores, recall_scores, f1_scores, support_scores = precision_recall_fscore_support(
        flat_true, flat_pred, labels=["BRAND", "TYPE", "VOLUME", "PERCENT", "O"], zero_division=0
    )

    # Создание таблицы
    table_data = []
    for label_name, prec, rec, f1, supp in zip(
        ["BRAND", "TYPE", "VOLUME", "PERCENT", "O"],
        precision_scores, recall_scores, f1_scores, support_scores
    ):
        table_data.append([
            label_name,
            f"{prec:.4f}",
            f"{rec:.4f}",
            f"{f1:.4f}",
            int(supp)
        ])

    # Агрегированные метрики
    macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(
        flat_true, flat_pred, average="macro", zero_division=0
    )
    micro_prec, micro_rec, micro_f1, _ = precision_recall_fscore_support(
        flat_true, flat_pred, average="micro", zero_division=0
    )
    weighted_prec, weighted_rec, weighted_f1, _ = precision_recall_fscore_support(
        flat_true, flat_pred, average="weighted", zero_division=0
    )

    # Добавление агрегированных метрик в таблицу
    table_data.append(["---", "---", "---", "---", "---"])
    table_data.append(["MACRO", f"{macro_prec:.4f}", f"{macro_rec:.4f}", f"{macro_f1:.4f}", "-"])
    table_data.append(["MICRO", f"{micro_prec:.4f}", f"{micro_rec:.4f}", f"{micro_f1:.4f}", "-"])
    table_data.append(["WEIGHTED", f"{weighted_prec:.4f}", f"{weighted_rec:.4f}", f"{weighted_f1:.4f}", "-"])

    # Заголовки таблицы
    headers = ["Класс", "Precision", "Recall", "F1-Score", "Support"]

    # Вывод таблицы
    print("📊 ДЕТАЛИЗИРОВАННЫЕ МЕТРИКИ МОДЕЛИ NER")
    print("=" * 65)
    print(tabulate(table_data, headers=headers, tablefmt="grid", stralign="center"))

    # Возвращаем также словарь с метриками для дальнейшего использования
    metrics_dict = {
        "class_metrics": {
            label: {"precision": prec, "recall": rec, "f1": f1, "support": supp}
            for label, prec, rec, f1, supp in zip(
                ["BRAND", "TYPE", "VOLUME", "PERCENT", "O"],
                precision_scores, recall_scores, f1_scores, support_scores
            )
        },
        "aggregate_metrics": {
            "macro": {"precision": macro_prec, "recall": macro_rec, "f1": macro_f1},
            "micro": {"precision": micro_prec, "recall": micro_rec, "f1": micro_f1},
            "weighted": {"precision": weighted_prec, "recall": weighted_rec, "f1": weighted_f1}
        }
    }

    return metrics_dict

In [24]:
# Запуск вычисления метрик с табличным выводом
print("🔍 Вычисление метрик на тестовом наборе...")
detailed_metrics = compute_detailed_metrics_table(ner_model, tokenized_test_set)

🔍 Вычисление метрик на тестовом наборе...


📊 ДЕТАЛИЗИРОВАННЫЕ МЕТРИКИ МОДЕЛИ NER
+----------+-------------+----------+------------+-----------+
|  Класс   |  Precision  |  Recall  |  F1-Score  |  Support  |
+==========+=============+==========+============+===========+
|  BRAND   |   0.4301    |  0.9004  |   0.5821   |   1596    |
+----------+-------------+----------+------------+-----------+
|   TYPE   |   0.9951    |  0.8224  |   0.9005   |   20699   |
+----------+-------------+----------+------------+-----------+
|  VOLUME  |   0.4773    |  0.7500  |   0.5833   |    28     |
+----------+-------------+----------+------------+-----------+
| PERCENT  |   0.0000    |  0.0000  |   0.0000   |     0     |
+----------+-------------+----------+------------+-----------+
|    O     |   0.0000    |  0.0000  |   0.0000   |     5     |
+----------+-------------+----------+------------+-----------+
|   ---    |     ---     |   ---    |    ---     |    ---    |
+----------+-------------+----------+------------+-----------+
|  MACRO   |   0.